![LetNet-4](image\Linear.jpg)

In [1]:
#Linear classifier

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO) #作用：将 TensorFlow 日志信息输出到屏幕

In [2]:
#定义模型函数
def Linear_model_fn(features, labels, mode):
  """Model function for LetNet4."""
  # Input Layer
  # Reshape X to 4-D tensor: [batch_size, width, height, channels]
  # MNIST images are 28x28 pixels, and have one color channel
  input_layer = tf.reshape(features["x"], [-1, 28*28])

  # Dense Layer
  # Densely connected layer with 10 neurons
  # Input Tensor Shape: [batch_size, 28*28]
  # Output Tensor Shape: [batch_size, 10]
  dense = tf.layers.dense(inputs=input_layer, units=10, activation=tf.nn.relu)

  # Add dropout operation; 0.6 probability that element will be kept
  #dropout = tf.layers.dropout(
  #    inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits layer
  # Input Tensor Shape: [batch_size, 10]
  # Output Tensor Shape: [batch_size, 10]
  logits = tf.layers.dense(inputs=dense, units=10)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }
  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    '''
    boundaries = [600*2, 600*5, 600*8, 600*12]
    values = [0.01, 0.001, 0.0001, 0.00005, 0.00001]
    global_step = tf.train.get_or_create_global_step()
    learning_rate = tf.train.piecewise_constant(global_step, boundaries, values)
    '''
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


In [3]:
#定义estimator
def main(unused_argv):
  # Load training and eval data
  mnist = tf.contrib.learn.datasets.load_dataset("mnist")
  train_data = mnist.train.images  # Returns np.array
  train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
  eval_data = mnist.test.images  # Returns np.array
  eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)
    
  # Create the Estimator
  logdir = "model/mnist_Linear_model"
  count = "0"
  Linear_classifier = tf.estimator.Estimator(
      model_fn=Linear_model_fn, model_dir=logdir + count)

  # Set up logging for predictions
  # Log the values in the "Softmax" tensor with label "probabilities"
  #tensors_to_log = {"probabilities": "softmax_tensor"}
  #logging_hook = tf.train.LoggingTensorHook(
  #    tensors=tensors_to_log, every_n_iter=500)
  
  # 运行10个Iterations
  for i in range(20):
      # Train the model
      train_input_fn = tf.estimator.inputs.numpy_input_fn(
          x={"x": train_data},
          y=train_labels,
          batch_size=100,
          num_epochs=None,
          shuffle=True)
      Linear_classifier.train(
          input_fn=train_input_fn,
          steps=600) #可加参数hooks=[logging_hook]

      # Evaluate the model and print results 
      eval_input_fn = tf.estimator.inputs.numpy_input_fn(
          x={"x": eval_data}, y=eval_labels, num_epochs=1, shuffle=False)
      eval_results = Linear_classifier.evaluate(input_fn=eval_input_fn)
      print(eval_results)


In [4]:
#运行程序
if __name__ == "__main__":
  tf.app.run()

Instructions for updating:
Please use tf.data.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data\train-labels-idx1-ubyte.gz
Extracting MNIST-data\t10k-images-idx3-ubyte.gz
Extracting MNIST-data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'model/mnist_Linear_model0', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_c

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model/mnist_Linear_model0\model.ckpt-13800
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 13800 into model/mnist_Linear_model0\model.ckpt.
INFO:tensorflow:loss = 0.22977556, step = 13801
INFO:tensorflow:global_step/sec: 478.974
INFO:tensorflow:loss = 0.23939537, step = 13901 (0.211 sec)
INFO:tensorflow:global_step/sec: 519.52
INFO:tensorflow:loss = 0.2705054, step = 14001 (0.193 sec)
INFO:tensorflow:global_step/sec: 511.568
INFO:tensorflow:loss = 0.23664436, step = 14101 (0.194 sec)
INFO:tensorflow:global_step/sec: 603.162
INFO:tensorflow:loss = 0.26002902, step = 14201 (0.164 sec)
INFO:tensorflow:global_step/sec: 495.81
INFO:tensorflow:loss = 0.29573146, step = 14301 (0.202 sec)
INFO:tensorflow:Saving checkpoints for 14400 into model/mnist_Linear_model

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-03-05-09:34:24
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model/mnist_Linear_model0\model.ckpt-16800
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-03-05-09:34:24
INFO:tensorflow:Saving dict for global step 16800: accuracy = 0.9212, global_step = 16800, loss = 0.276087
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 16800: model/mnist_Linear_model0\model.ckpt-16800
{'accuracy': 0.9212, 'loss': 0.276087, 'global_step': 16800}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model/mnist_Linear_model0\model.ckpt-16800
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoint

INFO:tensorflow:global_step/sec: 359.865
INFO:tensorflow:loss = 0.2984302, step = 19301 (0.278 sec)
INFO:tensorflow:global_step/sec: 541.207
INFO:tensorflow:loss = 0.22710843, step = 19401 (0.185 sec)
INFO:tensorflow:global_step/sec: 553.828
INFO:tensorflow:loss = 0.28844866, step = 19501 (0.181 sec)
INFO:tensorflow:global_step/sec: 541.671
INFO:tensorflow:loss = 0.26242083, step = 19601 (0.185 sec)
INFO:tensorflow:global_step/sec: 543.268
INFO:tensorflow:loss = 0.2626652, step = 19701 (0.185 sec)
INFO:tensorflow:Saving checkpoints for 19800 into model/mnist_Linear_model0\model.ckpt.
INFO:tensorflow:Loss for final step: 0.2339631.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-03-05-09:34:40
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model/mnist_Linear_model0\model.ckpt-19800
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished

INFO:tensorflow:Saving 'checkpoint_path' summary for global step 22200: model/mnist_Linear_model0\model.ckpt-22200
{'accuracy': 0.9225, 'loss': 0.27388206, 'global_step': 22200}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model/mnist_Linear_model0\model.ckpt-22200
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 22200 into model/mnist_Linear_model0\model.ckpt.
INFO:tensorflow:loss = 0.29926184, step = 22201
INFO:tensorflow:global_step/sec: 532.778
INFO:tensorflow:loss = 0.24677485, step = 22301 (0.198 sec)
INFO:tensorflow:global_step/sec: 547.516
INFO:tensorflow:loss = 0.33337668, step = 22401 (0.173 sec)
INFO:tensorflow:global_step/sec: 533.941
INFO:tensorflow:loss = 0.20906173, step = 22501 (0.189 sec)
INFO:tensorflow:global_step/sec: 560.968
INFO:tensorflow:loss

SystemExit: 

F:\Users\JackyPC\Anaconda3\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:3275: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


+ 关于`tf.app.run`有两种情况：

    + 如果你的代码中的入口函数不叫`main()`，而是一个其他名字的函数，如`test()`，则你应该这样写入口`tf.app.run(test())`
    + 如果你的代码中的入口函数叫`main()`，则你就可以把入口写成`tf.app.run()`
    
    
+ 在测试集上的准确率为92.25%，误差为7.75%，优于论文12%